<a href="https://colab.research.google.com/github/vishnuvryeruva/Latent-features--Manifolds/blob/main/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import time

names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

In [13]:
data = pd.read_csv("/content/drive/MyDrive/Data2/2018-personality-data.csv")
data

,userid,openness,agreeableness,emotional_stability,conscientiousness,extraversion,assigned metric,assigned condition,movie_1,predicted_rating_1,...,movie_9,predicted_rating_9,movie_10,predicted_rating_10,movie_11,predicted_rating_11,movie_12,predicted_rating_12,is_personalized,enjoy_watching
0,8e7cebf9a234c064b75016249f2ac65e,5.0,2.0,3.0,2.5,6.5,serendipity,high,77658,4.410466,...,120138,4.244817,121372,4.396004,127152,4.120456,95311,4.053847,4,4
1,77c7d756a093150d4377720abeaeef76,7.0,4.0,6.0,5.5,4.0,all,default,94959,4.207280,...,56782,4.019599,5618,3.963953,969,4.174188,1232,4.334877,2,3
2,b7e8a92987a530cc368719a0e60e26a3,4.0,3.0,4.5,2.0,2.5,serendipity,medium,110501,4.868064,...,2288,4.823212,3307,4.676756,1172,4.649281,1212,4.744990,2,2
3,92561f21446e017dd6b68b94b23ad5b7,5.5,5.5,4.0,4.5,4.0,popularity,medium,2905,4.526371,...,3030,4.425689,1281,4.479921,940,4.355061,905,4.317927,3,3
4,030001ac2145a938b07e686a35a2d638,5.5,5.5,3.5,4.5,2.5,popularity,medium,2905,4.526371,...,3030,4.425689,1281,4.479921,940,4.355061,905,4.317927,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1829,cff910b71f09b3120289ff6b461a9e03,5.5,3.5,2.5,4.0,5.5,popularity,low,108979,4.246346,...,6643,4.409108,115122,3.960470,7700,4.178546,67997,4.085300,3,3
1830,1ab3a4c2921d8da640854819b0f6cfce,4.0,3.5,4.5,4.0,2.5,serendipity,high,93040,4.227140,...,5618,4.149697,903,4.116152,38061,4.155210,1197,4.045751,3,4
1831,a06386edadf3bc614dadb7044708c46c,6.0,3.0,5.5,3.5,6.0,serendipity,low,106173,3.935297,...,26519,3.998642,89707,4.144870,2571,3.860041,108709,3.899857,3,4
1832,bad56d9506832cd79d874a6b66b3d813,5.0,3.5,1.5,3.5,2.5,serendipity,medium,6874,4.241766,...,92259,4.819710,32,4.059369,3730,4.427336,3435,4.844386,4,4


In [14]:
data[' agreeableness'] = round(((data[' agreeableness']*5)/7),1)
data[' openness'] = round(((data[' openness']*5)/7),1)
data[' emotional_stability'] = round(((data[' emotional_stability']*5)/7),1)
data[' conscientiousness'] = round(((data[' conscientiousness']*5)/7),1)
data[' extraversion'] = round(((data[' extraversion']*5)/7),1)
data

,userid,openness,agreeableness,emotional_stability,conscientiousness,extraversion,assigned metric,assigned condition,movie_1,predicted_rating_1,...,movie_9,predicted_rating_9,movie_10,predicted_rating_10,movie_11,predicted_rating_11,movie_12,predicted_rating_12,is_personalized,enjoy_watching
0,8e7cebf9a234c064b75016249f2ac65e,3.6,1.4,2.1,1.8,4.6,serendipity,high,77658,4.410466,...,120138,4.244817,121372,4.396004,127152,4.120456,95311,4.053847,4,4
1,77c7d756a093150d4377720abeaeef76,5.0,2.9,4.3,3.9,2.9,all,default,94959,4.207280,...,56782,4.019599,5618,3.963953,969,4.174188,1232,4.334877,2,3
2,b7e8a92987a530cc368719a0e60e26a3,2.9,2.1,3.2,1.4,1.8,serendipity,medium,110501,4.868064,...,2288,4.823212,3307,4.676756,1172,4.649281,1212,4.744990,2,2
3,92561f21446e017dd6b68b94b23ad5b7,3.9,3.9,2.9,3.2,2.9,popularity,medium,2905,4.526371,...,3030,4.425689,1281,4.479921,940,4.355061,905,4.317927,3,3
4,030001ac2145a938b07e686a35a2d638,3.9,3.9,2.5,3.2,1.8,popularity,medium,2905,4.526371,...,3030,4.425689,1281,4.479921,940,4.355061,905,4.317927,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1829,cff910b71f09b3120289ff6b461a9e03,3.9,2.5,1.8,2.9,3.9,popularity,low,108979,4.246346,...,6643,4.409108,115122,3.960470,7700,4.178546,67997,4.085300,3,3
1830,1ab3a4c2921d8da640854819b0f6cfce,2.9,2.5,3.2,2.9,1.8,serendipity,high,93040,4.227140,...,5618,4.149697,903,4.116152,38061,4.155210,1197,4.045751,3,4
1831,a06386edadf3bc614dadb7044708c46c,4.3,2.1,3.9,2.5,4.3,serendipity,low,106173,3.935297,...,26519,3.998642,89707,4.144870,2571,3.860041,108709,3.899857,3,4
1832,bad56d9506832cd79d874a6b66b3d813,3.6,2.5,1.1,2.5,1.8,serendipity,medium,6874,4.241766,...,92259,4.819710,32,4.059369,3730,4.427336,3435,4.844386,4,4


In [15]:
data.drop(['userid', ' movie_1', ' movie_2', ' movie_3', ' movie_4', ' movie_5', ' movie_6', ' movie_7',
            ' movie_8', ' movie_9', ' movie_10', ' movie_11', ' movie_12'],axis=1,inplace=True)
data

,openness,agreeableness,emotional_stability,conscientiousness,extraversion,assigned metric,assigned condition,predicted_rating_1,predicted_rating_2,predicted_rating_3,...,predicted_rating_5,predicted_rating_6,predicted_rating_7,predicted_rating_8,predicted_rating_9,predicted_rating_10,predicted_rating_11,predicted_rating_12,is_personalized,enjoy_watching
0,3.6,1.4,2.1,1.8,4.6,serendipity,high,4.410466,4.271995,4.611922,...,4.147292,4.098206,4.064843,4.149100,4.244817,4.396004,4.120456,4.053847,4,4
1,5.0,2.9,4.3,3.9,2.9,all,default,4.207280,4.266540,4.211322,...,4.090358,4.202424,4.113912,4.094422,4.019599,3.963953,4.174188,4.334877,2,3
2,2.9,2.1,3.2,1.4,1.8,serendipity,medium,4.868064,4.710444,5.029360,...,4.660769,4.962319,4.706864,4.645191,4.823212,4.676756,4.649281,4.744990,2,2
3,3.9,3.9,2.9,3.2,2.9,popularity,medium,4.526371,4.456451,4.668444,...,4.530360,4.292660,4.341634,4.261166,4.425689,4.479921,4.355061,4.317927,3,3
4,3.9,3.9,2.5,3.2,1.8,popularity,medium,4.526371,4.456451,4.668444,...,4.530360,4.292660,4.341634,4.261166,4.425689,4.479921,4.355061,4.317927,2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1829,3.9,2.5,1.8,2.9,3.9,popularity,low,4.246346,4.002232,3.981009,...,3.888507,4.368495,4.554531,4.145526,4.409108,3.960470,4.178546,4.085300,3,3
1830,2.9,2.5,3.2,2.9,1.8,serendipity,high,4.227140,4.069527,4.336292,...,4.217929,4.101192,4.244278,4.473696,4.149697,4.116152,4.155210,4.045751,3,4
1831,4.3,2.1,3.9,2.5,4.3,serendipity,low,3.935297,4.006561,3.947412,...,4.002666,4.235115,4.191760,3.965657,3.998642,4.144870,3.860041,3.899857,3,4
1832,3.6,2.5,1.1,2.5,1.8,serendipity,medium,4.241766,4.350788,4.399071,...,4.493639,4.397887,4.207513,4.212995,4.819710,4.059369,4.427336,4.844386,4,4


In [16]:
data.drop(data.loc[:, ' assigned metric':' is_personalized'].columns, axis=1, inplace=True)
data

,openness,agreeableness,emotional_stability,conscientiousness,extraversion,enjoy_watching
0,3.6,1.4,2.1,1.8,4.6,4
1,5.0,2.9,4.3,3.9,2.9,3
2,2.9,2.1,3.2,1.4,1.8,2
3,3.9,3.9,2.9,3.2,2.9,3
4,3.9,3.9,2.5,3.2,1.8,3
...,...,...,...,...,...,...
1829,3.9,2.5,1.8,2.9,3.9,3
1830,2.9,2.5,3.2,2.9,1.8,4
1831,4.3,2.1,3.9,2.5,4.3,4
1832,3.6,2.5,1.1,2.5,1.8,4


In [17]:
Personality = (data[' agreeableness'] + data[' conscientiousness'] + data[' emotional_stability'] + data[' extraversion'] + data[' openness'])
data['Personality'] = Personality
data = pd.DataFrame(data)
data

,openness,agreeableness,emotional_stability,conscientiousness,extraversion,enjoy_watching,Personality
0,3.6,1.4,2.1,1.8,4.6,4,13.5
1,5.0,2.9,4.3,3.9,2.9,3,19.0
2,2.9,2.1,3.2,1.4,1.8,2,11.4
3,3.9,3.9,2.9,3.2,2.9,3,16.8
4,3.9,3.9,2.5,3.2,1.8,3,15.3
...,...,...,...,...,...,...,...
1829,3.9,2.5,1.8,2.9,3.9,3,15.0
1830,2.9,2.5,3.2,2.9,1.8,4,13.3
1831,4.3,2.1,3.9,2.5,4.3,4,17.1
1832,3.6,2.5,1.1,2.5,1.8,4,11.5


In [18]:
a = data['Personality'].tolist()
b = [0] * len(a)
for i in range(0,1834):
  if a[i]<12.0:
    b[i]= "Bad" 
  elif a[i]>=12.0 and a[i]<17.0 :
    b[i] = "Medium" 
  else:
    b[i]= "Good"

data.head(5)

,openness,agreeableness,emotional_stability,conscientiousness,extraversion,enjoy_watching,Personality
0,3.6,1.4,2.1,1.8,4.6,4,13.5
1,5.0,2.9,4.3,3.9,2.9,3,19.0
2,2.9,2.1,3.2,1.4,1.8,2,11.4
3,3.9,3.9,2.9,3.2,2.9,3,16.8
4,3.9,3.9,2.5,3.2,1.8,3,15.3


In [19]:
d = pd.DataFrame(b)
d

,0
0,Medium
1,Good
2,Bad
3,Medium
4,Medium
...,...
1829,Medium
1830,Medium
1831,Good
1832,Bad


In [20]:
result = pd.concat([data, d], axis=1, join='inner')
result

,openness,agreeableness,emotional_stability,conscientiousness,extraversion,enjoy_watching,Personality,0
0,3.6,1.4,2.1,1.8,4.6,4,13.5,Medium
1,5.0,2.9,4.3,3.9,2.9,3,19.0,Good
2,2.9,2.1,3.2,1.4,1.8,2,11.4,Bad
3,3.9,3.9,2.9,3.2,2.9,3,16.8,Medium
4,3.9,3.9,2.5,3.2,1.8,3,15.3,Medium
...,...,...,...,...,...,...,...,...
1829,3.9,2.5,1.8,2.9,3.9,3,15.0,Medium
1830,2.9,2.5,3.2,2.9,1.8,4,13.3,Medium
1831,4.3,2.1,3.9,2.5,4.3,4,17.1,Good
1832,3.6,2.5,1.1,2.5,1.8,4,11.5,Bad


In [21]:
result.set_axis([*result.columns[:-1], 'Test'], axis=1, inplace=True)

In [22]:
result.drop_duplicates(inplace=True)

In [24]:
result['Test'].value_counts()

Medium    1124
Good       592
Bad         88
Name: Test, dtype: int64

In [25]:
le = preprocessing.LabelEncoder()
le.fit(result['Test'])

LabelEncoder()

In [26]:
result['Test'] = le.transform(result['Test'])
result

,openness,agreeableness,emotional_stability,conscientiousness,extraversion,enjoy_watching,Personality,Test
0,3.6,1.4,2.1,1.8,4.6,4,13.5,2
1,5.0,2.9,4.3,3.9,2.9,3,19.0,1
2,2.9,2.1,3.2,1.4,1.8,2,11.4,0
3,3.9,3.9,2.9,3.2,2.9,3,16.8,2
4,3.9,3.9,2.5,3.2,1.8,3,15.3,2
...,...,...,...,...,...,...,...,...
1829,3.9,2.5,1.8,2.9,3.9,3,15.0,2
1830,2.9,2.5,3.2,2.9,1.8,4,13.3,2
1831,4.3,2.1,3.9,2.5,4.3,4,17.1,1
1832,3.6,2.5,1.1,2.5,1.8,4,11.5,0


In [27]:
X = result.drop(['Test'], axis=1)
y = result['Test']

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

In [29]:
classifiers = [
    DecisionTreeClassifier(max_depth=3,random_state=17),
    RandomForestClassifier(max_depth=3, random_state=17, n_estimators=10),
    KNeighborsClassifier(n_neighbors=3),
    MLPClassifier(random_state=17,learning_rate_init=0.01,max_iter=250),
    GaussianProcessClassifier(random_state=17)
]

In [30]:
max_score = 0.0
max_class = ''
# iterate over classifiers
for name, clf in zip(names, classifiers):
    start_time = time.time()
    clf.fit(X_train, y_train)
    score = 100.0 * clf.score(X_test, y_test)
    print('Classifier = %s, Score (test, accuracy) = %.2f,' %(name, score), 'Training time = %.2f seconds' % (time.time() - start_time))
    
    if score > max_score:
        clf_best = clf
        max_score = score
        max_class = name

Classifier = Nearest Neighbors, Score (test, accuracy) = 100.00, Training time = 0.01 seconds
Classifier = Linear SVM, Score (test, accuracy) = 94.18, Training time = 0.11 seconds
Classifier = RBF SVM, Score (test, accuracy) = 94.46, Training time = 0.06 seconds
Classifier = Gaussian Process, Score (test, accuracy) = 95.84, Training time = 3.68 seconds
Classifier = Decision Tree, Score (test, accuracy) = 98.34, Training time = 8.49 seconds


In [31]:
print('Best --> Classifier = %s, Score (test, accuracy) = %.2f' %(max_class, max_score))

Best --> Classifier = Nearest Neighbors, Score (test, accuracy) = 100.00
